In [ ]:
#Question 2. 
''' Consider two dataset “Company.txt” and “College.txt”
    a. Create two table structure as follows by using case class:
        i. College table:
            dates:String,
            college:String,
            location:String,
            number_of_indian_students:String,
            number_of_foreign_students:String,
            fees:String , 
            survey_name:String, 
            rank:Int
        ii. Company table:
            dates:String,
            college:String,
            company:String,
            location:String,
            country:String,
            package:Double ,
            selected:Int, 
            participated:Int, 
            criteria:Int
            
    b. Read these two txt files into RDD and map the RDD with these table structure.
    c. Get the result year wise college ranking
    d. List down the College name and Total Numbers of Students including Indian & Foreigner who
    conducted the campus drive in “TCS” and “Capgemini”.
    e. List down the College name and ranking whose student have got opportunities in “Google” and
    “Facebook”
    f. List down the Company names with number of campus drives.
    g. List down the colleges state wise with total no. of Student Selection.
    h. List down the full details of College with Campus drive details who have highest number of Foreign
    Students.
    i. List down top 5 college name with their number of Selected Students
    j. List down the College Name with Fees whose student are getting package above of 10 LPA package
    k. List down the College Name with the Rank who have the highest criteria to get a job
'''

In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import col, year, to_date, split

In [2]:
spark = SparkSession.builder.appName("Spark Assessment2").getOrCreate()

24/04/24 14:30:04 WARN Utils: Your hostname, Shivanis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.106 instead (on interface en0)
24/04/24 14:30:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/24 14:30:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
College = StructType([
    StructField("dates", StringType(), True),
    StructField("college", StringType(), True),
    StructField("location", StringType(), True),
    StructField("number_of_indian_students", StringType(), True),
    StructField("number_of_foreign_students", StringType(), True),
    StructField("fees", StringType(), True),
    StructField("survey_name", StringType(), True),
    StructField("rank", IntegerType(), True)
])

Company = StructType([
    StructField("dates", StringType(), True),
    StructField("college", StringType(), True),
    StructField("company", StringType(), True),
    StructField("location", StringType(), True),
    StructField("country", StringType(), True),
    StructField("package", FloatType(), True),
    StructField("selected", IntegerType(), True),
    StructField("participated", IntegerType(), True),
    StructField("criteria", IntegerType(), True)
])

In [6]:
collegeRDD = spark.sparkContext.textFile("College.txt")
companyRDD = spark.sparkContext.textFile("Company.txt")

collegeRDD.collect()

collegeDF = spark.createDataFrame(collegeRDD.map(lambda x: x.split("|")), schema=College)
# collegeDF = collegeRDD.toDF()

# collegeDF = spark.createDataFrame(collegeRDD.map(lambda x: x.split("\t")), schema=College)
# companyDF = spark.createDataFrame(companyRDD.map(lambda x: x.split("\t")), schema=Company)

In [7]:
collegeDF.show()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=30767) is multi-threaded, use of fork() may lead to deadlocks in the child.
24/04/24 14:30:44 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 2)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch)

Py4JJavaError: An error occurred while calling o42.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 2) (192.168.1.106 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/

In [ ]:
collegeDF = collegeDF.withColumn("year", year(to_date(col("dates"), "yyyy-MM-dd")))
collegeRankingDF = collegeDF.groupBy("year", "college") \
    .agg({"rank": "avg"}) \
    .orderBy("year", "avg(rank)")

In [ ]:
from pyspark.sql import functions as F

companyDF = companyDF.filter(col("company").isin(["TCS", "Capgemini"]))
collegeStudentCountDF = companyDF.join(collegeDF, on="college", how="inner") \
    .groupBy("college") \
    .agg((F.col("number_of_indian_students").cast("int") + F.col("number_of_foreign_students").cast("int")).alias("total_students")) \
    .select("college", "total_students")

In [ ]:
companyDriveCountDF = companyDF.groupBy("company").count() \
    .select("company", "count") \
    .orderBy("count", ascending=False)

In [ ]:
collegeDF = collegeDF.withColumn("state", split(col("location"), ",")[1])  # Assuming comma-separated location format
selectedStudentsDF = companyDF.join(collegeDF, on="college", how="inner") \
    .filter(col("selected") > 0)
stateStudentCountDF = selectedStudentsDF.groupBy("state") \
    .agg(col("selected").sum().alias("total_selections")) \
    .orderBy("total_selections", ascending=False)

In [ ]:
maxForeignStudents = collegeDF.select(col("number_of_foreign_students").cast("int")).rdd.max()[0]
collegeWithMaxForeignStudentsDF = collegeDF.filter(col("number_of_foreign_students").cast("int") == maxForeignStudents)
joinedDetailsDF = collegeWithMaxForeignStudentsDF.join(companyDF, on="college", how="full")  # Full join to include colleges without drives

In [ ]:
topSelectedStudentsDF = companyDF.join(collegeDF, on="college", how="inner") \
    .groupBy("college") \
    .agg(col("selected").sum().alias("total_selected")) \
    .orderBy("total_selected", ascending=False) \
    .limit(5)

In [ ]:
packageAbove10DF = companyDF.join(collegeDF, on="college", how="inner") \
    .filter(col("package") > 10) \
    .select("college", "fees")

In [ ]:
maxCriteria = companyDF.select(col("criteria").cast("int")).rdd.max()[0]
collegeWithMaxCriteriaDF = collegeDF.filter(col("criteria") == maxCriteria)

In [ ]:
collegeDF.printSchema()
companyDF.printSchema()